In [17]:
# !pip install rusenttokenize
# !pip install tokenizers

В этом семинаре мы попробуем натренировать простую версию GPT. Код в очень большой части основан на вот этот туториале от Andrey Karpathy - https://www.youtube.com/watch?v=kCc8FmEb1nY . Я добавил более объемный датасет, BPE токенизацию, а также некоторые дополнительные пояснения, но основной код практически точно такой же, потому что сложно придумать что-то лучше.

Помимо собственно разбора GPT модели, в этом семинаре также разбирается pytorch. 
Но прежде чем переходить к этому, давайте загрузим данные.

## Данные и токенизация

Для обучения GPT нужен текст. Чем больше, тем лучше. В туториале использовался просто корпус текстов Шекспира, но я решил взять чуть более реалистичный и объемный текст - новостные тексты.

In [1]:
import pandas as pd
from rusenttokenize import ru_sent_tokenize

In [2]:
data = pd.read_csv('https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/data/lenta_40k.csv.zip')

In [3]:
data.head()

,text,topic
0,Россия должна сотрудничать с Всемирным антидоп...,Спорт
1,Уголовный суд Кувейта 28 июня освободил под за...,Мир
2,Французский журнал Charlie Hebdo опубликовал н...,Интернет и СМИ
3,В Петербурге в доме № 53 по улице Лени Голиков...,Россия
4,"В московском аэропорту ""Домодедово"" задержан г...",Россия


Нам понадобятся только тексты. Также трансформеры очень чувствительны к длине текста, поэтому для простоты разделим все тексты на предложения и будем считать каждое предложение одним текстом.

In [5]:
sentences = []
for text in data.text.values:
    sentences.extend(ru_sent_tokenize(text))

In [6]:
# почти 500 тыс предложений
len(sentences)

489727

In [7]:
# cохраним в отдельный файл чтобы больше не тратить время на токенизацию,
# также файл понадобится дальше для обучения токенизатора
f = open('corpus.txt', 'w')
for sent in sentences:
    f.write(sent + '\n')
f.close()

In [8]:
sentences = open('corpus.txt').read().splitlines()

Теперь нам нужно обучить токенизатор. В туториале Karpathy обучал GPT на символах, но обычно используется subword токенизация. Чтобы сделать эту модель чуть менее игрушечной, давайте добавим такую токенизацию. На саму модель это никак не повлияет, так как она в любом случае ожидает список индексов как input. С символьной токенизацией такие списки будут очень длинным, а subword токенизация группирует символы в нграммы (или даже целые слова) и таким образом длина последовательности сокращается.

Мы не будем писать алгоритм токенизации с нуля, а воспользуемся готовым решением от huggingface - библиотекой tokenizers. Она написана на Rust и поэтому достаточно быстрая. 
Токенизация будет основана на алгоритме Byte-Pair-Encoding. В нем строки сначала кодируются как байты, которые представляют индекс символа в таблице Юникода. И в процессе обучения отдельные байты группируются по частотности в нграммы до тех пор, пока размер словаря не достигнет заданого лимита. 
По умолчанию это значение - 30 тысяч. Также в словаре также обычно добавляют какие-то специальные токены, которых нет в текстах, но которые будут добавляться, чтобы передать в модели какие-то дополнительные параметры. Мы добавим три токена - паддинг, токен начала текста, токена конца текста. Паддинг будет использоваться чтобы сравнять все тексты до одной длины при передаче в модель, но мы не хотим, чтобы эти токены как-то влияли на обучение, поэтому их нужно будет замаскировать. А BOS и EOS мы уже использовали в простых языковых моделях. Тут они нужны для тех же целей. В реальных моделях также в специальные токены входят токены разделяющие части промпта - системное сообщение, сообщение пользователя, сообщение модели и т.п. Про них мы поговорим в следующих семинарах.

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer(BPE()) 
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"])

In [46]:
# токенайзер обучается на файле а не на питоновских списках
tokenizer.train(files=["corpus.txt"], trainer=trainer)

In [47]:
# сохраним токенизатор
tokenizer.save('tokenizer')

In [11]:
# при перезапуске можно просто перезагрузить готовый токенизатор
# также он понадобится если мы решим сохранить модель
tokenizer = Tokenizer.from_file("tokenizer")

In [12]:
vocab_size = tokenizer.get_vocab_size()

Посмотрим что получается в результате токенизации

In [14]:
tokenizer.encode('Какой-то текст').ids

[702, 660, 15, 393, 7975]

Напишем еще функцию которая будет подставлять BOS и EOS токены

In [15]:
def encode(text, tokenizer):
    return [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[EOS]')]

Индекс паддинг токена пригодится позже для маскинга

In [16]:
PAD_IDX = tokenizer.token_to_id('[PAD]')

Теперь токенизируем все предложения и создадим датасет, который будет передавать в модель

In [17]:
import torch

In [18]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, sentences, tokenizer, max_len=32):
        # каждое предложение преобразуется в последовательность индексов 
        # а списки преобразуются в тензоры
        self.encoded_texts = [torch.LongTensor(encode(sent, tokenizer)[-max_len:]) for sent in sentences]
        # чтобы составить один общий обучающий тензор нужно сравнять длины последовательностей отдельных текстов
        # в торче не такая удобная функция паддинга, поэтому транкация (отрезание лишнего) происходит уже выше
        self.X = torch.nn.utils.rnn.pad_sequence(self.encoded_texts, padding_value=PAD_IDX, batch_first=True)
        self.length = len(self.encoded_texts)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # обучающий пример для GPT составляется из одного текста
        # x - это все токены кроме последнего
        # y - это все токены кроме первого
        # другими словами, y это x со сдвигом вправо
        # каждый отдельный элемент в y - следующий токен для соответствующего элемента в x
        # tokens = [1,2,3,4,5]
        # x = [1,2,3,4]
        # y = [2,3,4,5]
        
        x = self.X[index][:-1]
        y = self.X[index][1:]
        
        # чтобы не учитывать паддинг нам нужно создать маску
        mask = x!=PAD_IDX

        return x, y, mask

Разделим данные на обучающие и валидационные (90% и 10%)

In [19]:
n = int(0.9*len(sentences)) # first 90% will be train, rest val
sentences_train = sentences[:n]
sentences_val = sentences[n:]

In [20]:
MAX_LEN = 64

In [21]:
training_set = Dataset(sentences_train, tokenizer, MAX_LEN)
val_set = Dataset(sentences_val, tokenizer, MAX_LEN)

In [22]:
training_set[0]

(tensor([    1,  1481,  2499, 13595,   226,  4896,   533, 12335, 26891, 14188,
            10, 14988,   807,  1045,   774, 19648,  3317,  4676, 11158,   389,
          8412,  9819,  1984,   521,   211,  9880,    15,   422,    15, 12185,
            16,     2,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 tensor([ 1481,  2499, 13595,   226,  4896,   533, 12335, 26891, 14188,    10,
         14988,   807,  1045,   774, 19648,  3317,  4676, 11158,   389,  8412,
          9819,  1984,   521,   211,  9880,    15,   422,    15, 12185,    16,
             2,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     

In [23]:
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )
val_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=False)

Теперь можно переходить к обучению. Сначала давайте посмотрим на отдельные инструменты pytorch, которые будут использовать в моделях

#### torch 

In [24]:
# в коде часто встречается вот такой импорт
import torch
import torch.nn as nn
import torch.nn.functional as F

Торч во многом похож на numpy и в нем есть те же стандартные функции 

In [25]:
# создает тензор заданного размера и заполняет его нулями
torch.zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [26]:
# создает тензор заданного размера и заполняет его единицами
torch.ones((2, 3, 1))

tensor([[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.]]])

In [27]:
# создает тензор заданного размера и заполняет его случайными числами 0-1
torch.rand((3, 1, 2))

tensor([[[0.8246, 0.7336]],

        [[0.0444, 0.0924]],

        [[0.5371, 0.2244]]])

In [28]:
# размерность можно получить также как в numpy
t = torch.rand((3, 1, 2))
t.shape

torch.Size([3, 1, 2])

In [29]:
# чтобы изменить размерность можно использовать .view
t.view(3,2)

tensor([[0.8901, 0.6534],
        [0.3813, 0.0028],
        [0.2018, 0.4946]])

In [30]:
t.view(6)

tensor([0.8901, 0.6534, 0.3813, 0.0028, 0.2018, 0.4946])

In [31]:
# изменение размерности не добавляет и не удаляет данные из изначального тензора
# поэтому размеры должны совпадать
t.view(3,4)

RuntimeError: shape '[3, 4]' is invalid for input of size 6

In [ ]:
# если размерности нужно поменять местами то для этого используется transpose
# размерность dim1 меняется местами с размерностью dim2
t.transpose(-2, -1).shape

In [ ]:
# как и в numpy у тензоров есть специальный атрибут .T который возвращает транспонированный тензор
# но при обучении моделей мы обычно работает с тензорами, где первая размерность это batch_size и ее нужно оставить на своей месте
# поэтому используется transpose c указанием конкретных размерностей а batch размерность остается на своем месте
t.T

In [32]:
# с помощью транспонирования можно рассчитать dot product (=близость) между
# векторами в последовательности в батче
# t размерности (4,3,2) и напрямую умножить его на себя нельзя потому что размерности 1,2 и 1,2 не подходят (нужно чтобы внутрение сходились)
# transpose(-2, -1) или transpose(2, 1) что то же самое 
# позволят рассчитать dot product чем простое умножение @
# оно тоже уже умеет работать с батчами поэтому первая разверность не изменится
# в результате размерность будет 4,3,3
# то есть для каждого примера в батче размером 4
# между всеми элементами последовательности размером 3
# будет рассчита dot product между отдельными векторами размерности 2
# близость считается между парами в последовательности поэтому в результате для каждого будет 3 близости
t = torch.rand((4, 3, 2))
(t @ t.transpose(-2, -1)).shape, (t @ t.transpose(2, 1)).shape

(torch.Size([4, 3, 3]), torch.Size([4, 3, 3]))

In [33]:
# если одна из размерностей единичная то ее можно схлопнуть 
t = torch.rand((4, 1, 2))
t, t.squeeze(1), t.squeeze(1).shape

(tensor([[[0.8008, 0.9708]],
 
         [[0.1924, 0.2296]],
 
         [[0.1065, 0.2852]],
 
         [[0.8307, 0.0645]]]),
 tensor([[0.8008, 0.9708],
         [0.1924, 0.2296],
         [0.1065, 0.2852],
         [0.8307, 0.0645]]),
 torch.Size([4, 2]))

In [34]:
# также можно добавить единичную размерность 
t, t.unsqueeze(3), t.unsqueeze(3).shape

(tensor([[[0.8008, 0.9708]],
 
         [[0.1924, 0.2296]],
 
         [[0.1065, 0.2852]],
 
         [[0.8307, 0.0645]]]),
 tensor([[[[0.8008],
           [0.9708]]],
 
 
         [[[0.1924],
           [0.2296]]],
 
 
         [[[0.1065],
           [0.2852]]],
 
 
         [[[0.8307],
           [0.0645]]]]),
 torch.Size([4, 1, 2, 1]))

In [35]:
t1 = torch.rand((1,3,4))
t2 = torch.rand((2,3,4))

In [36]:
# cat конкатенирует тензоры по заданной размерности (по умолчанию первой - то есть батч размерности)
torch.cat([t1, t2]).shape

torch.Size([3, 3, 4])

In [37]:
# для создания масок для attention понадобится функция tril которая зануляет элементы выше диагонали 
torch.tril(torch.ones((2,3,3)))

tensor([[[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]],

        [[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]]])

In [38]:
# c помощью tril и функции masked_fill можно заменять значения в тензоре по диагональному паттерну

In [39]:
t = torch.rand((4, 3, 3))
t

tensor([[[0.7635, 0.4642, 0.7521],
         [0.4594, 0.7915, 0.5790],
         [0.0320, 0.3731, 0.9466]],

        [[0.6682, 0.3126, 0.8954],
         [0.9256, 0.7902, 0.4135],
         [0.1221, 0.3147, 0.1836]],

        [[0.2292, 0.6574, 0.7839],
         [0.2278, 0.7917, 0.1904],
         [0.9878, 0.6356, 0.5606]],

        [[0.7482, 0.4246, 0.8443],
         [0.9940, 0.1193, 0.7985],
         [0.0143, 0.4395, 0.3698]]])

In [40]:
# в тезоре t значения которые соответствуют нулю в треугольной маске заменяются на минус бесконечность
t.masked_fill(torch.tril(torch.ones((3,3)))==0, float("-inf"))

tensor([[[0.7635,   -inf,   -inf],
         [0.4594, 0.7915,   -inf],
         [0.0320, 0.3731, 0.9466]],

        [[0.6682,   -inf,   -inf],
         [0.9256, 0.7902,   -inf],
         [0.1221, 0.3147, 0.1836]],

        [[0.2292,   -inf,   -inf],
         [0.2278, 0.7917,   -inf],
         [0.9878, 0.6356, 0.5606]],

        [[0.7482,   -inf,   -inf],
         [0.9940, 0.1193,   -inf],
         [0.0143, 0.4395, 0.3698]]])

In [134]:
# с помощью такой операции в GPT реализован механизм внимания где каждый токен общается только с токенами до него

### deep learning layers torch

In [142]:
# Embedding слой сопоставляет вектор индексу
# первый аргумент - размерность словаря
# второй - размерность вектора
embed = nn.Embedding(10, 20)

In [141]:
# два текста по 4 токена в каждом
t = torch.LongTensor([[1,3,4,5], [3,4,5,6]])

In [145]:
t.shape

torch.Size([2, 4])

In [144]:
# в результате каждому токену сопоставляется вектор 20
embed(t).shape

torch.Size([2, 4, 20])

In [151]:
# полносвязный слой или линейной преобразование
# первый аргумент изначальная размерность
# второй - выходная размерность
linear = nn.Linear(2, 10)

In [152]:
# изначально у нас есть batch с двумя примерами где каждый состоит из трех токенов и у каждого токена вектор 2
t = torch.rand((2, 3, 2))

In [153]:
# применив полносвязный слой мы получим то же самое только размерность векторов теперь 10
linear(t).shape

torch.Size([2, 3, 10])

In [154]:
# функция активации которая зануляет отрицательные значения
relu = nn.ReLU()

In [156]:
linear(t), relu(linear(t))

(tensor([[[-0.5116, -0.5753,  0.1465,  0.1994,  0.5395, -0.0600, -0.0829,
            0.0278, -0.5628, -0.5003],
          [-0.5552, -1.0051,  0.1534,  0.2507,  0.2597, -0.3272, -0.2982,
           -0.0311, -0.2502, -0.2841],
          [-0.5205, -0.9500,  0.2155,  0.4156,  0.2760, -0.3582, -0.4242,
            0.1563, -0.2666, -0.1669]],
 
         [[-0.5201, -0.6877,  0.1545,  0.2296,  0.4644, -0.1363, -0.1543,
            0.0301, -0.4787, -0.4296],
          [-0.4676, -0.5273,  0.2303,  0.4240,  0.5444, -0.1179, -0.2653,
            0.2761, -0.5658, -0.3297],
          [-0.5000, -0.8396,  0.2337,  0.4569,  0.3417, -0.3102, -0.4177,
            0.2286, -0.3394, -0.1764]]], grad_fn=<ViewBackward0>),
 tensor([[[0.0000, 0.0000, 0.1465, 0.1994, 0.5395, 0.0000, 0.0000, 0.0278,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.1534, 0.2507, 0.2597, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.2155, 0.4156, 0.2760, 0.0000, 0.0000, 0.1563,
          

In [159]:
# softmax - функция активации которая нормализует значения в векторе так что они лежат в интервале от 0-1 и суммируются в 1
linear(t), F.softmax(linear(t), dim=-1)

(tensor([[[-0.5116, -0.5753,  0.1465,  0.1994,  0.5395, -0.0600, -0.0829,
            0.0278, -0.5628, -0.5003],
          [-0.5552, -1.0051,  0.1534,  0.2507,  0.2597, -0.3272, -0.2982,
           -0.0311, -0.2502, -0.2841],
          [-0.5205, -0.9500,  0.2155,  0.4156,  0.2760, -0.3582, -0.4242,
            0.1563, -0.2666, -0.1669]],
 
         [[-0.5201, -0.6877,  0.1545,  0.2296,  0.4644, -0.1363, -0.1543,
            0.0301, -0.4787, -0.4296],
          [-0.4676, -0.5273,  0.2303,  0.4240,  0.5444, -0.1179, -0.2653,
            0.2761, -0.5658, -0.3297],
          [-0.5000, -0.8396,  0.2337,  0.4569,  0.3417, -0.3102, -0.4177,
            0.2286, -0.3394, -0.1764]]], grad_fn=<ViewBackward0>),
 tensor([[[0.0643, 0.0603, 0.1242, 0.1309, 0.1840, 0.1010, 0.0987, 0.1103,
           0.0611, 0.0650],
          [0.0663, 0.0423, 0.1348, 0.1485, 0.1499, 0.0833, 0.0858, 0.1121,
           0.0900, 0.0870],
          [0.0647, 0.0421, 0.1350, 0.1649, 0.1434, 0.0761, 0.0712, 0.1272,
          

In [161]:
# LayerNorm нормализует значения в векторах так что среднее равно 0 а стандартное отклонение 1
ln = nn.LayerNorm(10)

In [163]:
out = linear(t)

In [171]:
out.mean(), out.std()

(tensor(-0.1457, grad_fn=<MeanBackward0>),
 tensor(0.3879, grad_fn=<StdBackward0>))

In [172]:
ln(out).mean(), ln(out).std()

(tensor(-7.9473e-09, grad_fn=<MeanBackward0>),
 tensor(1.0084, grad_fn=<StdBackward0>))

In [173]:
out, ln(out)

(tensor([[[-0.5116, -0.5753,  0.1465,  0.1994,  0.5395, -0.0600, -0.0829,
            0.0278, -0.5628, -0.5003],
          [-0.5552, -1.0051,  0.1534,  0.2507,  0.2597, -0.3272, -0.2982,
           -0.0311, -0.2502, -0.2841],
          [-0.5205, -0.9500,  0.2155,  0.4156,  0.2760, -0.3582, -0.4242,
            0.1563, -0.2666, -0.1669]],
 
         [[-0.5201, -0.6877,  0.1545,  0.2296,  0.4644, -0.1363, -0.1543,
            0.0301, -0.4787, -0.4296],
          [-0.4676, -0.5273,  0.2303,  0.4240,  0.5444, -0.1179, -0.2653,
            0.2761, -0.5658, -0.3297],
          [-0.5000, -0.8396,  0.2337,  0.4569,  0.3417, -0.3102, -0.4177,
            0.2286, -0.3394, -0.1764]]], grad_fn=<ViewBackward0>),
 tensor([[[-1.0233, -1.1978,  0.7791,  0.9240,  1.8556,  0.2135,  0.1508,
            0.4542, -1.1635, -0.9925],
          [-0.9336, -2.1461,  0.9759,  1.2380,  1.2624, -0.3192, -0.2410,
            0.4787, -0.1118, -0.2032],
          [-0.8853, -1.9469,  0.9337,  1.4283,  1.0833, -0.4841, 

In [174]:
# dropout зануляет случайные значения в векторах
# параметр задает вероятность зануления
dropout = nn.Dropout(0.5)

In [175]:
dropout(linear(t))

tensor([[[-1.0232, -1.1506,  0.0000,  0.0000,  0.0000, -0.1201, -0.1659,
           0.0000, -1.1255, -1.0007],
         [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,
          -0.0000, -0.0000, -0.0000],
         [-1.0410, -0.0000,  0.4310,  0.8312,  0.5521, -0.7164, -0.0000,
           0.0000, -0.5332, -0.3338]],

        [[-1.0403, -1.3754,  0.3089,  0.0000,  0.9288, -0.0000, -0.0000,
           0.0602, -0.0000, -0.8591],
         [-0.0000, -0.0000,  0.0000,  0.8481,  0.0000, -0.0000, -0.0000,
           0.5523, -1.1317, -0.0000],
         [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.6205, -0.0000,
           0.0000, -0.0000, -0.0000]]], grad_fn=<MulBackward0>)

In [176]:
# Sequential позволяет соединить несколько слоев в одно последовательное преобразование
net = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, 2),
            nn.Dropout(0.5),
        )

In [179]:
out = net(t)

In [180]:
out.shape

torch.Size([2, 3, 2])

In [181]:
out

tensor([[[-0.2389,  0.0000],
         [-0.0000,  0.8493],
         [-0.4012,  0.0000]],

        [[-0.0000,  0.7760],
         [-0.2761,  0.8136],
         [-0.3664,  0.8802]]], grad_fn=<MulBackward0>)

## GPT

Теперь давайте разберем код Karpath в котором он собирает GPT и обучим ее на наших данных.


GPT это трансформерная модель. Она реализована также как это описано в статье Attention is all you need, за исключением того, что в GPT есть только decoder (правый столб). Статья AIAYN изначально про машинный перевод и поэтому в ней используется encoder-decoder архитектура. 
Декодером GPT делает то, что в ней используется causual attention, где каждый токен общается только с предыдущими. В encoder все токены взаимодействуют со всеми.
Также небольшое отличие состоит в порядке применения layerNorm (сейчас его перенесли до MHA и до FF)



![](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F0235fd2f-26f4-47ff-b95e-eddf6a4593b0_782x1152.png)

In [41]:
# hyperparameters
block_size = MAX_LEN # what is the maximum context length for predictions?
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 64 # размерность эмбеддингов и векторов внутри трансформера
#ffn_hid_dim = n_embd * 4
n_head = 4
n_layer = 4
dropout = 0.0

In [42]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        if mask is not None:
            wei.masked_fill(~mask.unsqueeze(1), float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        out = torch.cat([h(x, mask) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, inp):
        # print(inp)
        x, mask = inp
        x = x + self.sa(self.ln1(x), mask)
        x = x + self.ffwd(self.ln2(x))
        return (x, mask)

In [49]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None, mask=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x, mask = self.blocks((x, mask)) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=PAD_IDX)

        return logits, loss

    def generate(self, idx, max_new_tokens, stop_token):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)# (B, 1)
            if idx_next == stop_token:
                break
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [50]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

4.073392 M parameters


In [51]:
# model

In [52]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Код обучения просто передает в модель батчи из датасета

In [53]:
def train(model, iterator, optimizer, print_every=10):
    epoch_loss = []
    model.train()  

    for i, (xs, ys, mask) in enumerate(iterator):
        optimizer.zero_grad()   
        logits, loss = model(xs.to(device), ys.to(device), mask.to('cuda')) 
        
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {torch.Tensor(epoch_loss).mean(-1)}')
        
    return torch.Tensor(epoch_loss).mean(-1)

def evaluate(model, iterator):
    epoch_loss = []
    model.eval()  
    with torch.no_grad():
        for xs, ys, mask in iterator:   
            logits, loss = model(xs.to(device), ys.to(device), mask.to('cuda'))     
            epoch_loss.append(loss.item())  
            
    return torch.Tensor(epoch_loss).mean(-1)

При обучении после каждой эпохи генерируется текст чтобы видеть прогресс

In [ ]:
train_losses = []
eval_losses = []
for i in range(30):
    print(i)
    train_losses.append(train(model, training_generator, optimizer, 100))
    eval_loss = evaluate(model, val_generator)
    print('Eval - ', eval_loss.item())
    eval_losses.append(eval_loss)
    pred = model.generate(torch.LongTensor([[tokenizer.token_to_id('[BOS]')]]).to('cuda'), 200, tokenizer.token_to_id('[EOS]'))
    print(tokenizer.decode_batch(pred.detach().cpu().numpy()))

0
Loss: 8.453104019165039
Loss: 8.051098823547363
